In [1]:
%%writefile mjkey.txt
MuJoCo Pro Individual license activation key, number 7777, type 6.

Issued to Everyone.

Expires October 18, 2031.

Do not modify this file. Its entire content, including the
plain text section, is used by the activation manager.

9aaedeefb37011a8a52361c736643665c7f60e796ff8ff70bb3f7a1d78e9a605
0453a3c853e4aa416e712d7e80cf799c6314ee5480ec6bd0f1ab51d1bb3c768f
8c06e7e572f411ecb25c3d6ef82cc20b00f672db88e6001b3dfdd3ab79e6c480
185d681811cfdaff640fb63295e391b05374edba90dd54cc1e162a9d99b82a8b
ea3e87f2c67d08006c53daac2e563269cdb286838b168a2071c48c29fedfbea2
5effe96fe3cb05e85fb8af2d3851f385618ef8cdac42876831f095e052bd18c9
5dce57ff9c83670aad77e5a1f41444bec45e30e4e827f7bf9799b29f2c934e23
dcf6d3c3ee9c8dd2ed057317100cd21b4abbbf652d02bf72c3d322e0c55dcc24

Writing mjkey.txt


In [2]:
%%bash
wget -q https://www.roboti.us/download/mjpro150_linux.zip
unzip -q -d ~/.mujoco mjpro150_linux.zip
wget -q https://www.roboti.us/download/mujoco200_linux.zip
unzip -q -d ~/.mujoco mujoco200_linux.zip
ln -s ~/.mujoco/mujoco200_linux ~/.mujoco/mujoco200
wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz
tar -xf mujoco210-linux-x86_64.tar.gz -C ~/.mujoco/
cp mjkey.txt ~/.mujoco/

In [3]:
import os

os.environ['LD_LIBRARY_PATH'] += ":/root/.mujoco/mujoco210/bin"

In [4]:
!git clone https://github.com/jiangsy/mbpo_pytorch

Cloning into 'mbpo_pytorch'...
remote: Enumerating objects: 907, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 907 (delta 96), reused 79 (delta 79), pack-reused 803 (from 1)
Receiving objects: 100% (907/907), 165.83 KiB | 4.48 MiB/s, done.
Resolving deltas: 100% (596/596), done.


In [5]:
%%writefile mbpo_pytorch/mbpo_pytorch/envs/benchmarking_envs/gym/half_cheetah.py
import os

import numpy as np
from gym import utils
from gym.envs.mujoco import mujoco_env

from mbpo_pytorch.envs.virtual_env import BaseModelBasedEnv


class HalfCheetahEnv(mujoco_env.MujocoEnv, utils.EzPickle, BaseModelBasedEnv):

    def __init__(self, frame_skip=5):
        self.prev_qpos = None
        dir_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
        mujoco_env.MujocoEnv.__init__(
            self, '%s/assets/half_cheetah.xml' % dir_path, frame_skip=frame_skip
        )
        utils.EzPickle.__init__(self)

    def step(self, action: np.ndarray):
        start_ob = self._get_obs()
        reward_run = start_ob[8]

        self.do_simulation(action, self.frame_skip)
        ob = self._get_obs()
        if getattr(self, 'action_space', None):
            action = np.clip(action, self.action_space.low,
                             self.action_space.high)
        reward_ctrl = -0.1 * np.square(action).sum()

        reward = reward_run + reward_ctrl
        done = False
        return ob, reward, done, {}

    def _get_obs(self):
        return np.concatenate([
            self.sim.data.qpos.flat[1:],
            self.sim.data.qvel.flat,
        ])

    def mb_step(self, states, actions, next_states):
        # returns rewards and dones
        # forward rewards are calculated based on states, instead of next_states as in original SLBO envs
        if getattr(self, 'action_space', None):
            actions = np.clip(actions, self.action_space.low,
                              self.action_space.high)
        rewards = - self.cost_np_vec(states, actions, next_states)
        return rewards, np.zeros_like(rewards, dtype=bool)

    def reset_model(self):
        qpos = self.init_qpos + \
            self.np_random.uniform(low=-.1, high=.1, size=self.model.nq)
        qvel = self.init_qvel + self.np_random.randn(self.model.nv) * .1
        self.set_state(qpos, qvel)
        return self._get_obs()

    def viewer_setup(self):
        self.viewer.cam.distance = self.model.stat.extent * 0.5

    def cost_np_vec(self, obs, acts, next_obs):
        reward_ctrl = -0.1 * np.sum(np.square(acts), axis=1)
        reward_run = obs[:, 8]
        reward = reward_run + reward_ctrl
        return -reward

    def cost_tf_vec(self, obs, acts, next_obs):
        raise NotImplementedError

Overwriting mbpo_pytorch/mbpo_pytorch/envs/benchmarking_envs/gym/half_cheetah.py


In [6]:
%%writefile mbpo_pytorch/mbpo_pytorch/thirdparty/dummy_vec_env.py

from collections import OrderedDict
import numpy as np
from typing import Sequence

from mbpo_pytorch.thirdparty.base_vec_env import VecEnv
from mbpo_pytorch.thirdparty.util import copy_obs_dict, dict_to_obs, obs_space_info


class DummyVecEnv(VecEnv):
    """
    Creates a simple vectorized wrapper for multiple environments, calling each environment in sequence on the current
    Python process. This is useful for computationally simple environment such as ``cartpole-v1``, as the overhead of
    multiprocess or multithread outweighs the environment computation time. This can also be used for RL methods that
    require a vectorized environment, but that you want a single environments to train with.

    :param env_fns: ([callable]) A list of functions that will create the environments
        (each callable returns a `Gym.Env` instance when called).
    """

    def __init__(self, env_fns):
        self.envs = [fn() for fn in env_fns]
        env = self.envs[0]
        VecEnv.__init__(self, len(env_fns), env.observation_space, env.action_space)
        obs_space = env.observation_space
        self.keys, shapes, dtypes = obs_space_info(obs_space)

        self.buf_obs = OrderedDict([
            (k, np.zeros((self.num_envs,) + tuple(shapes[k]), dtype=dtypes[k]))
            for k in self.keys])
        self.buf_dones = np.zeros((self.num_envs,), dtype=bool)
        self.buf_rews = np.zeros((self.num_envs,), dtype=np.float32)
        self.buf_infos = [{} for _ in range(self.num_envs)]
        self.actions = None
        self.metadata = env.metadata

    def step_async(self, actions):
        self.actions = actions

    def step_wait(self):
        for env_idx in range(self.num_envs):
            obs, self.buf_rews[env_idx], self.buf_dones[env_idx], self.buf_infos[env_idx] =\
                self.envs[env_idx].step(self.actions[env_idx])
            if self.buf_dones[env_idx]:
                # save final observation where user can get it, then reset
                self.buf_infos[env_idx]['terminal_observation'] = obs
                obs = self.envs[env_idx].reset()
            self._save_obs(env_idx, obs)
        return (self._obs_from_buf(), np.copy(self.buf_rews), np.copy(self.buf_dones),
                self.buf_infos.copy())

    def seed(self, seed=None):
        seeds = list()
        for idx, env in enumerate(self.envs):
            seeds.append(env.seed(seed + idx))
        return seeds

    def reset(self):
        for env_idx in range(self.num_envs):
            obs = self.envs[env_idx].reset()
            self._save_obs(env_idx, obs)
        return self._obs_from_buf()

    def close(self):
        for env in self.envs:
            env.close()

    def get_images(self, *args, **kwargs) -> Sequence[np.ndarray]:
        return [env.render(*args, mode='rgb_array', **kwargs) for env in self.envs]

    def render(self, *args, **kwargs):
        """
        Gym environment rendering. If there are multiple environments then
        they are tiled together in one image via `BaseVecEnv.render()`.
        Otherwise (if `self.num_envs == 1`), we pass the render call directly to the
        underlying environment.

        Therefore, some arguments such as `mode` will have values that are valid
        only when `num_envs == 1`.

        :param mode: The rendering type.
        """
        if self.num_envs == 1:
            return self.envs[0].render(*args, **kwargs)
        else:
            return super().render(*args, **kwargs)

    def _save_obs(self, env_idx, obs):
        for key in self.keys:
            if key is None:
                self.buf_obs[key][env_idx] = obs
            else:
                self.buf_obs[key][env_idx] = obs[key]

    def _obs_from_buf(self):
        return dict_to_obs(self.observation_space, copy_obs_dict(self.buf_obs))

    def get_attr(self, attr_name, indices=None):
        """Return attribute from vectorized environment (see base class)."""
        target_envs = self._get_target_envs(indices)
        return [getattr(env_i, attr_name) for env_i in target_envs]

    def set_attr(self, attr_name, value, indices=None):
        """Set attribute inside vectorized environments (see base class)."""
        target_envs = self._get_target_envs(indices)
        for env_i in target_envs:
            setattr(env_i, attr_name, value)

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        """Call instance methods of vectorized environments."""
        target_envs = self._get_target_envs(indices)
        return [getattr(env_i, method_name)(*method_args, **method_kwargs) for env_i in target_envs]

    def _get_target_envs(self, indices):
        indices = self._get_indices(indices)
        return [self.envs[i] for i in indices]

Overwriting mbpo_pytorch/mbpo_pytorch/thirdparty/dummy_vec_env.py


In [31]:
%%writefile mbpo_pytorch/mbpo_pytorch/configs/config.py
import os

import argparse
import munch
import yaml
from yaml import Loader
import collections.abc

from mbpo_pytorch.misc import logger


def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.abc.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, str(v)))
    return dict(items)


def safe_eval(exp: str):
    try:
        return eval(exp)
    except (NameError, SyntaxError, TypeError):
        return exp


def deflatten_with_eval(d, sep='.'):
    deflattend_d = {}
    for k, v in d.items():
        d = deflattend_d
        key_seq = k.split(sep)
        for key in key_seq[:-1]:
            try:
                d = d[key]
            except (TypeError, KeyError):
                d[key] = {}
                d = d[key]
        d[key_seq[-1]] = safe_eval(v)
    return deflattend_d


class Config:
    def __new__(cls, config_paths='config.yaml'):
        parser = argparse.ArgumentParser()
        parser.add_argument('--configs', nargs='+', default=[])
        parser.add_argument('--set', type=str, nargs='*', action='append')

        args, unknown = parser.parse_known_args()
        flattened_config_dict = {}
        overwritten_config_dict = {}

        if args.configs:
            config_paths = args.configs

        if isinstance(config_paths, str):
            config_paths = [config_paths]

        for config_path in config_paths:
            if not config_path.startswith('/'):
                config_path = os.path.join(os.path.dirname(__file__), config_path)
            logger.info('Loading configs from {}.'.format(config_path))

            with open(config_path, 'r', encoding='utf-8') as f:
                new_config_dict = yaml.load(f, Loader=Loader)
                flattened_new_config_dict = flatten(new_config_dict)
                overwritten_config_dict.update(
                    {k: v for k, v in flattened_new_config_dict.items()
                     if (k in flattened_config_dict.keys() and v != flattened_config_dict[k])})
                flattened_config_dict.update(flattened_new_config_dict)

        if args.set:
            for instruction in sum(args.set, []):
                key, value = instruction.split('=')
                flattened_config_dict.update({key: safe_eval(value)})
                # values set by args should be recorded all
                overwritten_config_dict.update({key: safe_eval(value)})

        config_dict = deflatten_with_eval(flattened_config_dict)

        for key, value in overwritten_config_dict.items():
            logger.notice('Hyperparams {} has been overwritten to {}.'.format(key, value))

        config = munch.munchify(config_dict)
        config_dict = flatten(config_dict)
        logged_config_dict = {}

        for key, value in config_dict.items():
            if key.find('.') >= 0:
                logged_config_dict[key] = value
        return config, logged_config_dict

Overwriting mbpo_pytorch/mbpo_pytorch/configs/config.py


In [8]:
%%writefile mbpo_pytorch/mbpo_pytorch/scripts/run_mbpo.py
import time
from collections import deque

import numpy as np
import torch
from operator import itemgetter

from mbpo_pytorch.algos import MBPO, SAC
from mbpo_pytorch.configs.config import Config
from mbpo_pytorch.envs.wrapped_envs import make_vec_envs, make_vec_virtual_envs
from mbpo_pytorch.misc import logger
from mbpo_pytorch.misc.utils import set_seed, get_seed, log_and_write, evaluate, commit_and_save, init_logging
from mbpo_pytorch.models import Actor, QCritic, RunningNormalizer, EnsembleRDynamics
from mbpo_pytorch.storages import SimpleUniversalBuffer as Buffer, MixtureBuffer


def main():
    config, hparam_dict = Config(['mbpo.yaml', 'halfcheetah.yaml'])
    set_seed(config.seed)
    commit_and_save(config.proj_dir, config.save_dir, False)

    writer, log_dir, eval_log_dir, save_dir = init_logging(config, hparam_dict)

    mf_config = config.sac
    mb_config = config.mbpo

    device = torch.device(config.device)

    real_envs = make_vec_envs(config.env.env_name, get_seed(), config.env.num_real_envs, config.env.gamma,
                              log_dir, device, allow_early_resets=True, norm_reward=False, norm_obs=False)

    state_dim = real_envs.observation_space.shape[0]
    action_space = real_envs.action_space
    action_dim = action_space.shape[0]

    datatype = {'states': {'dims': [state_dim]}, 'next_states': {'dims': [state_dim]},
                'actions': {'dims': [action_dim]}, 'rewards': {'dims': [1]}, 'masks': {'dims': [1]}}

    state_normalizer = RunningNormalizer(state_dim)
    action_normalizer = RunningNormalizer(action_dim)
    state_normalizer.to(device)
    action_normalizer.to(device)

    dynamics = EnsembleRDynamics(state_dim, action_dim, 1, config.mbpo.dynamics_hidden_dims,
                                 mb_config.num_dynamics_networks, mb_config.num_elite_dynamics_networks,
                                 state_normalizer, action_normalizer)
    dynamics.to(device)

    actor = Actor(state_dim, action_space, mf_config.actor_hidden_dims, state_normalizer=None,
                  use_limited_entropy=False, use_tanh_squash=True, use_state_dependent_std=True)
    actor.to(device)

    q_critic1 = QCritic(state_dim, action_space, mf_config.critic_hidden_dims)
    q_critic2 = QCritic(state_dim, action_space, mf_config.critic_hidden_dims)
    q_critic_target1 = QCritic(state_dim, action_space, mf_config.critic_hidden_dims)
    q_critic_target2 = QCritic(state_dim, action_space, mf_config.critic_hidden_dims)
    q_critic1.to(device)
    q_critic2.to(device)
    q_critic_target1.to(device)
    q_critic_target2.to(device)

    target_entropy = mf_config.target_entropy or -np.prod(real_envs.action_space.shape).item()

    agent = SAC(actor, q_critic1, q_critic2, q_critic_target1, q_critic_target2, mf_config.batch_size,
                mf_config.num_grad_steps, config.env.gamma, 1.0, mf_config.actor_lr, mf_config.critic_lr,
                mf_config.soft_target_tau, target_entropy=target_entropy)

    virtual_envs = make_vec_virtual_envs(config.env.env_name, dynamics, get_seed(), 0, config.env.gamma, device,
                                         use_predicted_reward=True)

    base_virtual_buffer_size = mb_config.rollout_batch_size * config.env.max_episode_steps * \
                               mb_config.num_model_retain_epochs // mb_config.model_update_interval

    virtual_buffer = Buffer(base_virtual_buffer_size, datatype)
    virtual_buffer.to(device)
    agent.check_buffer(virtual_buffer)

    model = MBPO(dynamics, mb_config.dynamics_batch_size, rollout_schedule=mb_config.rollout_schedule, verbose=1,
                 lr=mb_config.lr, l2_loss_coefs=config.mbpo.l2_loss_coefs, max_num_epochs=mb_config.max_num_epochs)

    real_buffer = Buffer(mb_config.real_buffer_size, datatype)
    real_buffer.to(device)
    model.check_buffer(real_buffer)

    if mb_config.real_sample_ratio > 0:
        policy_buffer = MixtureBuffer([virtual_buffer, real_buffer],
                                      [(1 - mb_config.real_sample_ratio), mb_config.real_sample_ratio])
    else:
        policy_buffer = virtual_buffer

    assert not ((config.model_load_path is None) ^ (config.buffer_load_path is None)), 'Partial loading may cause bugs'

    if config.model_load_path:
        state_dicts = torch.load(config.model_load_path)
        dynamics_sd, actor_sd, q_critic1_sd, q_critic2_sd, q_critic_target1_sd, q_critic_target2_sd = \
            itemgetter('dynamics', 'actor', 'q_critic1', 'q_critic2', 'q_critic_target1', 'q_critic_target2')(
                state_dicts)
        dynamics.load_state_dict(dynamics_sd)
        actor.load_state_dict(actor_sd)
        q_critic1.load_state_dict(q_critic1_sd)
        q_critic2.load_state_dict(q_critic2_sd)
        q_critic_target1.load_state_dict(q_critic_target1_sd)
        q_critic_target2.load_state_dict(q_critic_target2_sd)

    if config.buffer_load_path:
        # virtual buffer does not need loading
        real_buffer.load(config.buffer_load_path)

    real_states = real_envs.reset()

    real_episode_rewards = deque(maxlen=30)
    real_episode_lengths = deque(maxlen=30)

    for _ in range(mb_config.num_warmup_samples):
        real_actions = torch.tensor([real_envs.action_space.sample() for _ in range(config.env.num_real_envs)]).to(
            device)
        real_next_states, real_rewards, real_dones, real_infos = real_envs.step(real_actions)
        real_masks = torch.tensor([[0.0] if done else [1.0] for done in real_dones], dtype=torch.float32)
        real_buffer.insert(states=real_states, actions=real_actions, rewards=real_rewards, masks=real_masks,
                           next_states=real_next_states)
        real_states = real_next_states

        real_episode_rewards.extend([info['episode']['r'] for info in real_infos if 'episode' in info])
        real_episode_lengths.extend([info['episode']['l'] for info in real_infos if 'episode' in info])

    recent_states, recent_actions = itemgetter('states', 'actions') \
        (real_buffer.get_recent_samples(mb_config.num_warmup_samples - mb_config.model_update_interval))

    state_normalizer.update(recent_states)
    action_normalizer.update(recent_actions)

    start = time.time()

    for epoch in range(config.mbpo.num_total_epochs):
        logger.info('Epoch {}:'.format(epoch))

        model.update_rollout_length(epoch)

        last_time = time.time()

        for i in range(config.env.max_episode_steps):
            losses = {}
            if i % mb_config.model_update_interval == 0:
                recent_states, recent_actions = itemgetter('states', 'actions') \
                    (real_buffer.get_recent_samples(mb_config.model_update_interval))
                state_normalizer.update(recent_states)
                action_normalizer.update(recent_actions)

                losses.update(model.update(real_buffer))
                initial_states = next(real_buffer.get_batch_generator_inf(mb_config.rollout_batch_size))['states']
                new_virtual_buffer_size = base_virtual_buffer_size * model.num_rollout_steps
                virtual_buffer.resize(new_virtual_buffer_size)
                model.generate_data(virtual_envs, virtual_buffer, initial_states, actor)

            with torch.no_grad():
                real_actions = actor.act(real_states)['actions']

            real_next_states, real_rewards, real_dones, real_infos = real_envs.step(real_actions)
            real_masks = torch.tensor([[0.0] if done else [1.0] for done in real_dones], dtype=torch.float32)
            real_buffer.insert(states=real_states, actions=real_actions, rewards=real_rewards, masks=real_masks,
                               next_states=real_next_states)
            real_states = real_next_states
            real_episode_rewards.extend([info['episode']['r'] for info in real_infos if 'episode' in info])
            real_episode_lengths.extend([info['episode']['l'] for info in real_infos if 'episode' in info])

            losses.update(agent.update(policy_buffer))

            if i % 100 == 0:
                logger.info('Time for last 100 iterations: {}:'.format((time.time() - last_time)))
                last_time = time.time()

            # only keys with '/' will be recorded in the tensorboard
            if i % config.log_interval == 0:
                time_elapsed = time.time() - start
                num_env_steps = epoch * config.env.max_episode_steps + i + mb_config.num_warmup_samples
                log_infos = [('/time_elapsed', time_elapsed), ('samples_collected', num_env_steps)]

                if len(real_episode_rewards) > 0:
                    log_infos.extend([('perf/ep_rew_real', np.mean(real_episode_rewards)),
                                      ('perf/ep_len_real', np.mean(real_episode_lengths))])
                for loss_name, loss_value in losses.items():
                    log_infos.append(('loss/' + loss_name, loss_value))
                log_and_write(writer, log_infos, global_step=num_env_steps)

        if (epoch + 1) % config.eval_interval == 0:
            episode_rewards_real_eval, episode_lengths_real_eval = \
                evaluate(actor, config.env.env_name, get_seed(), 10, eval_log_dir,
                         device, norm_reward=False, norm_obs=False)
            log_infos = [('perf/ep_rew_real_eval', np.mean(episode_rewards_real_eval)),
                         ('perf/ep_len_real_eval', np.mean(episode_lengths_real_eval))]
            log_and_write(writer, log_infos,
                          global_step=(epoch + 1) * config.env.max_episode_steps + mb_config.num_warmup_samples)

        if (epoch + 1) % config.save_interval == 0:
            state_dicts = {'dynamics': dynamics.state_dict(), 'actor': actor.state_dict(),
                           'q_critic1': q_critic1.state_dict(), 'q_critic2': q_critic2.state_dict(),
                           'q_critic_target1': q_critic_target1.state_dict(),
                           'q_critic_target2': q_critic_target2.state_dict()}
            torch.save(state_dicts, save_dir + '/state_dicts.pt')
            real_buffer.save(save_dir + '/real_buffer.pt')

    real_envs.close()
    virtual_envs.close()
    writer.close()


if __name__ == '__main__':
    main()

Overwriting mbpo_pytorch/mbpo_pytorch/scripts/run_mbpo.py


In [27]:
!sudo apt-get -qq -y install wget unzip software-properties-common \
    libosmesa6-dev patchelf libglew-dev

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
Selecting previously unselected package libglu1-mesa:amd64.
(Reading database ... 129783 files and directories currently installed.)
Preparing to unpack .../libglu1-mesa_9.0.2-1_amd64.deb ...
Unpacking libglu1-mesa:amd64 (9.0.2-1) ...
Selecting previously unselected package libglu1-mesa-dev:amd64.
Preparing to unpack .../libglu1-mesa-dev_9.0.2-1_amd64.deb ...
Unpacking libglu1-mesa-dev:amd64 (9.0.2-1) ...
Selecting previously unselected package libglew-dev:amd64.
Preparing to unpack .../libglew-dev_2.2.0-4_amd64.deb ...
Unpacking libglew-dev:amd64 (2.2.0-4) ...
Setting up libglu1-mesa:amd64 (9.0.2-1) ...
Setting up libglu1-mesa-dev:amd64 (9.0.2-1) ...
Setting up libglew-dev:amd64 (2.2.0-4) ...
Processing triggers for libc-bin (2.35-

In [10]:
!sudo apt install python3.8 python3-pip python3.8-distutils python3.8-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  mailcap mime-support python3-setuptools python3-wheel python3.8-lib2to3
  python3.8-minimal
Suggested packages:
  python-setuptools-doc python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  mailcap mime-support python3-pip python3-setuptools python3-wheel python3.8
  python3.8-dev python3.8-distutils python3.8-lib2to3 python3.8-minimal
0 upgraded, 14 newly installed, 0 to remove and 129 not upgraded.
Need to get 13.8 MB of archives.
After this operation, 54.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66

In [11]:
!python3.8 -m pip install setuptools==65.5.0 "wheel<0.40.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.9 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.6.0
    Uninstalling setuptools-59.6.0:
      Successfully uninstalled setuptools-59.6.0


In [12]:
!python3.8 -m pip install stable-baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 29.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 55.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 KB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 77.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 KB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
!pip install cython==0.29.37 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.11
    Uninstalling Cython-3.0.11:
      Successfully uninstalled Cython-3.0.11


In [15]:
!sudo apt-get install build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 129 not upgraded.


In [16]:
!python3.8 -m pip install --upgrade pip setuptools wheel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 KB 7.6 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: pip
    Found existing installation: pip 22.0.2
    Uninstalling pip-22.0.2:
      Successfully uninstalled pip-22.0.2


In [18]:
!sudo apt install python3-pip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-pip is already the newest version (22.0.2+dfsg-1ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 129 not upgraded.


In [20]:
!cd mbpo_pytorch && pip install -e .

Obtaining file:///kaggle/working/mbpo_pytorch
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 15.8 MB/s eta 0:00:00
  Created wheel for pyglib: filename=pyglib-0.1-py2.py3-none-any.whl size=4530 sha256=24eb89c914232a06ac35c2d3ac957e638c969cebe3de95395588b8fe9cce176c
  Stored in directory: /root/.cache/pip/wheels/3d/40/74/f2d023081eff881e653d4fe966b0f8603ae85e189062317f8f
  Created wheel for python-gflags: filename=python_gflags-3.1.2-py3-none-any.whl size=57370 sha256=2aaeb2fc1d74d03100ebd8fc7e4d647a3a76c917262433de279d0521b9d5970d
  Stored in directory: /root/.cache/pip/wheels/0d/7d/a2/e858f0632653a3c752a807f85a39ca9d7ed75796c62845e66c
Successfully built pyglib python-gflags
  Running setu

In [29]:
!pip install mujoco_py mujoco numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 57.2 MB/s eta 0:00:0000:0100:01


In [51]:
!python3.8 -m pip install gym==0.23.1 --force-reinstall

  Using cached gym-0.23.1.tar.gz (626 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
  Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Using cached importlib_metadata-8.5.0-py3-none-any.whl (26 kB)
  Using cached zipp-3.20.2-py3-none-any.whl (9.2 kB)
  Created wheel for gym: filename=gym-0.23.1-py3-none-any.whl size=701354 sha256=4cce6b77d7bd20c7720300e33a420b5a66a5c80a34b826ee88556b52d17546b1
  Stored in directory: /root/.cache/pip/wheels/78/28/77/b0c74e80a2a4faae0161d5c53bc4f8e436e77aedc79136ee13
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [31]:
!python3.8 -m pip install numpy

  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)


In [33]:
!pip install gym==0.23.1 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.2/626.2 kB 9.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.9 MB/s eta 0:00:00:00:010:01
  Created wheel for gym: filename=gym-0.23.1-py3-none-any.whl size=701456 sha256=1a1874fe56d2fe238bbcafe2949c0829b1823b7c7e908dce22daef4018e304c0
  Stored in directory: /root/.cache/pip/wheels/1a/00/fb/fe5cf2860fb9b7bc860e28f00095a1f42c7b726dd6f42d1acc
Successfully built gym
  Attempting uninstall: gym_notices
    Found existing installation: gym-notices 0.0.8
    Uninstalling gym-notices-0.0.8:
      Successfully uninstalled gym-notices-0.0.8
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Atte

In [ ]:
!python ./mbpo_pytorch/mbpo_pytorch/scripts/run_mbpo.py --set seed=0 verbose=1 device="cuda:0" env.env_name='HalfCheetah'

/usr/local/lib/python3.10/dist-packages/Cython/Distutils/old_build_ext.py:15: DeprecationWarning: dep_util is Deprecated. Use functions from setuptools instead.
  from distutils.dep_util import newer, newer_group
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
2025-03-30 10:51:55.387140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-30 10:51:55.409358: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-30 10:51:55.416980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regi